In [34]:
import warnings
warnings.filterwarnings('ignore')
import pandas as pd
import numpy as np
from dateutil.relativedelta import relativedelta
from datetime import datetime
from dateutil.parser import parse
import copy

from imblearn.over_sampling import SMOTE ,RandomOverSampler
#시각화
import matplotlib.pyplot as plt
import seaborn as sns


from sklearn.model_selection import train_test_split
from sklearn.model_selection import KFold, StratifiedKFold
from sklearn.metrics import *
from sklearn.metrics import accuracy_score, confusion_matrix, log_loss

##알고리즘 
from sklearn.ensemble import RandomForestClassifier
from xgboost import XGBClassifier
from lightgbm import LGBMClassifier

In [2]:
# 다양한 오차 측정 지표를 확인하기 위한 함수 정의

from sklearn.metrics import *

def get_clf_eval(y_test, y_data, pred=None, pred_proba=None): 
    confusion = confusion_matrix(y_test, pred) 
    accuracy = accuracy_score(y_test, pred) 
    precision = precision_val(y_data)
    recall = recall_val(y_data) 
    f1 = f1_score(y_test, pred , average= 'weighted') 
    return [accuracy, precision, recall, f1]

In [3]:
def Cut(Series , cuts):
    "구분할 iter가능한 변수와 구분 기준을 입력받으면 구분 기준 앞에서 부터 1로 구분해서 return 함"
    Cuts = copy.deepcopy(cuts)
    Cuts.append(np.inf)
    Cuts.insert(0,-1*np.inf)
    R = len(Cuts)
    label = range(1,R)
    return pd.cut(Series, Cuts, labels = label)

In [4]:
def precision_val(y_data):
    

    new = y_data[y_data['예측 y 라벨링'] > 2]
    real = new[new['실제 y라벨링'] > 2]
    if len(new) == 0:
        per = 0
    else :
        per = len(real)/len(new)
    return per

In [5]:
def recall_val(y_data):

    real = y_data[y_data['실제 y라벨링'] > 2]
    new = real[real['예측 y 라벨링'] > 2]
    if len(new) == 0:
        per = 0
    else :
        per = len(new)/len(real)
    return per

In [6]:
df_1_3 = pd.read_csv("d:/공모전/IPO/IPO/CleanData/X_train_After_IPO_1.csv",encoding = "cp949")
df_3_6 = pd.read_csv("d:/공모전/IPO/IPO/CleanData/X_train_After_IPO_3.csv",encoding = "cp949")
df_0 = pd.read_csv("d:/공모전/IPO/IPO/CleanData/X_train_Before_IPO.csv",encoding = "cp949")
df_end = pd.read_csv("d:/공모전/IPO/IPO/CleanData/X_train_EndPrice_IPO.csv",encoding = "cp949")

In [7]:
df_0_1 = df_0.drop(['공모가 대비 3개월 수익률','공모가 대비 6개월 수익률'],axis =1)
df_0_3 = df_0.drop(['공모가 대비 1개월 수익률','공모가 대비 6개월 수익률'],axis =1)
df_0_6 = df_0.drop(['공모가 대비 1개월 수익률','공모가 대비 3개월 수익률'],axis =1)

In [8]:
df_end_1 = df_end.drop(['종가 대비 3개월 수익률','종가 대비 6개월 수익률'],axis =1)
df_end_3 = df_end.drop(['종가 대비 1개월 수익률','종가 대비 6개월 수익률'],axis =1)
df_end_6 = df_end.drop(['종가 대비 1개월 수익률','종가 대비 3개월 수익률'],axis =1)

# 랜덤포레스트 값

In [47]:
def process_final(df,y_name):
    
    ## 기본 전처리
    df['상장일'] = pd.to_datetime(df['상장일'])
    df = df.set_index(['상장일'])
    df = df.drop(['종목명','공모 시가총액'],axis = 1) ## 나중에 카테고리 진행할려면 남겨줄 것
    if y_name == '공모가 대비 6개월 수익률' or "종가 대비 6개월 수익률":
        df['Cat'] = Cut(df[y_name],[ -0.2, 0.2, 0.4])
       
        
    else :
        df['Cat'] = Cut(df[y_name],[ -0.1, 0.1, 0.2])
       
    
    ## train 3년 test 1년으로 총 기간을 3개월 이동으로 36개 구간분할
    train_list = []
    test_list = []
    train_start_date = '2009-04-01' ## 기한은 나중에 변경할수도
    test_start_date = parse(str(train_start_date)).date() + relativedelta(years =4)
    train_end_date = parse(str(test_start_date)).date() - relativedelta(days =1)
    test_end_date = parse(str(train_end_date)).date() + relativedelta(years =1) 

    while True:

        train_list.append(df[train_start_date : train_end_date])
        test_list.append(df[test_start_date : test_end_date])

        train_start_date = parse(str(train_start_date)).date() + relativedelta(months=3)
        test_start_date = parse(str(test_start_date)).date() + relativedelta(months=3)
        train_end_date = parse(str(train_end_date)).date() + relativedelta(months=3)
        test_end_date = parse(str(test_end_date)).date() + relativedelta(months=3)

        if str(train_start_date) == '2017-04-01':
            break
            
    y_data = []
    score_list = []
    feature_list = []

    for i in range(0,len(train_list)):
        X_train_before = train_list[i].drop([y_name,'Cat'],axis =1)
        y_train_before = train_list[i]['Cat']
        X_test = test_list[i].drop([y_name,'Cat'],axis =1)
        y_test = test_list[i]['Cat']
        
        smote = SMOTE(random_state=0,k_neighbors = 3)
        X_train,y_train = smote.fit_resample(X_train_before,y_train_before)
#         ros = RandomOverSampler()
#         X_train,y_train = ros.fit_resample(X_train_before,y_train_before)

       # 랜덤 포레스트 학습 및 별도의 테스트 셋으로 예측 성능 평가
              
        clf = RandomForestClassifier(random_state=0 , max_depth = 5 , min_samples_leaf = 6 , min_samples_split =  12 , \
                                     n_estimators = 20 )
        

        clf.fit(X_train , y_train)
        train_pred = clf.predict(X_train_before)
        test_pred = clf.predict(X_test)




        #feature_importance
        feature_importance = clf.feature_importances_


        pred_value = pd.Series(test_pred,index = y_test.index)
        per = test_list[i][y_name]
        y_testdata = pd.concat([per,y_test,pred_value] , axis = 1)
        y_testdata.columns = ['실제 y 수익률','실제 y라벨링','예측 y 라벨링']
        
        train_pred_value = pd.Series(train_pred,index = y_train_before.index)
        per = train_list[i][y_name]
        y_traindata = pd.concat([per,y_train_before,train_pred_value] , axis = 1)
        y_traindata.columns = ['실제 y 수익률','실제 y라벨링','예측 y 라벨링']
        
        
        # 성과 평가
        train_res = get_clf_eval(y_train_before, y_traindata, train_pred)
        test_res = get_clf_eval(y_test, y_testdata,test_pred)
        res = train_res + test_res
        
        score_list.append(res)
        feature_list.append(feature_importance)
        y_data.append(y_testdata)   
    df_score = pd.DataFrame(score_list,columns = ["정확도","정밀도","재현율","f1_score"]+["정확도_test","정밀도_test","재현율_test","f1_score_test"]).T 
    df_score.rename(columns = lambda x : "model_set_"+ str(x),inplace = True)

    df_feature = pd.DataFrame(feature_list,columns = X_train.columns).T 
    df_feature.rename(columns = lambda x : "model_set_"+ str(x),inplace = True)
        
    return df_score , df_feature , y_data

In [48]:
df_score_0_1,df_feature_0_1,y_data_0_1 = process_final(df_0_1,'공모가 대비 1개월 수익률')

In [49]:
df_score_0_1

,model_set_0,model_set_1,model_set_2,model_set_3,model_set_4,model_set_5,model_set_6,model_set_7,model_set_8,model_set_9,...,model_set_22,model_set_23,model_set_24,model_set_25,model_set_26,model_set_27,model_set_28,model_set_29,model_set_30,model_set_31
정확도,0.696429,0.772727,0.693333,0.793750,0.785235,0.755245,0.812950,0.704698,0.798561,0.691781,...,0.670782,0.661157,0.680162,0.656904,0.606557,0.699571,0.626609,0.594828,0.620968,0.633333
정밀도,0.777778,0.736842,0.625000,0.779661,0.792453,0.793103,0.903846,0.791045,0.835616,0.786667,...,0.708333,0.647059,0.685714,0.608696,0.548387,0.677419,0.535714,0.534884,0.577236,0.644231
재현율,0.803279,0.857143,0.800000,0.836364,0.763636,0.851852,0.854545,0.791045,0.897059,0.808219,...,0.708333,0.767442,0.808989,0.853659,0.829268,0.875000,0.845070,0.932432,0.806818,0.705263
f1_score,0.701234,0.772427,0.697279,0.795504,0.785384,0.757600,0.812885,0.704192,0.797079,0.701044,...,0.674187,0.669780,0.694016,0.664932,0.633691,0.710082,0.639498,0.605550,0.633644,0.638768
정확도_test,0.200000,0.333333,0.444444,0.326087,0.355556,0.357143,0.377049,0.267606,0.311688,0.343284,...,0.354839,0.483871,0.476190,0.416667,0.328125,0.189655,0.246377,0.461538,0.400000,0.473684
정밀도_test,0.714286,0.562500,0.523810,0.548387,0.538462,0.538462,0.529412,0.434783,0.402985,0.395833,...,0.272727,1.000000,0.333333,0.444444,0.481481,0.548387,0.705882,0.677966,0.771429,0.736842
재현율_test,0.294118,0.562500,0.687500,0.653846,0.560000,0.700000,0.562500,0.937500,0.818182,0.730769,...,0.166667,0.055556,0.266667,0.222222,0.481481,0.472222,0.250000,0.888889,0.900000,1.000000
f1_score_test,0.209524,0.309604,0.415482,0.291540,0.325499,0.324970,0.376155,0.124883,0.239394,0.352981,...,0.311780,0.404765,0.458501,0.373737,0.344501,0.198686,0.260926,0.343699,0.389335,0.338346


In [12]:
df_score_0_1['rlf_mean'] = df_score_0_1.mean(axis=1)

In [13]:
rlf_score = df_score_0_1['rlf_mean']

In [14]:
rlf_score

정확도              0.702678
정밀도              0.726528
재현율              0.805926
f1_score         0.709760
정확도_test         0.322484
정밀도_test         0.476976
재현율_test         0.558717
f1_score_test    0.304779
Name: rlf_mean, dtype: float64

# xgb

In [15]:
def process_final_xgb(df,y_name):
    
    ## 기본 전처리
    df['상장일'] = pd.to_datetime(df['상장일'])
    df = df.set_index(['상장일'])
    df = df.drop(['종목명','공모 시가총액'],axis = 1) ## 나중에 카테고리 진행할려면 남겨줄 것
    if y_name == '공모가 대비 6개월 수익률' or "종가 대비 6개월 수익률":
        df['Cat'] = Cut(df[y_name],[ -0.2, 0.2, 0.4])
       
        
    else :
        df['Cat'] = Cut(df[y_name],[ -0.1, 0.1, 0.2])
       
    
    ## train 3년 test 1년으로 총 기간을 3개월 이동으로 36개 구간분할
    train_list = []
    test_list = []
    train_start_date = '2009-04-01' ## 기한은 나중에 변경할수도
    test_start_date = parse(str(train_start_date)).date() + relativedelta(years =4)
    train_end_date = parse(str(test_start_date)).date() - relativedelta(days =1)
    test_end_date = parse(str(train_end_date)).date() + relativedelta(years =1) 

    while True:

        train_list.append(df[train_start_date : train_end_date])
        test_list.append(df[test_start_date : test_end_date])

        train_start_date = parse(str(train_start_date)).date() + relativedelta(months=3)
        test_start_date = parse(str(test_start_date)).date() + relativedelta(months=3)
        train_end_date = parse(str(train_end_date)).date() + relativedelta(months=3)
        test_end_date = parse(str(test_end_date)).date() + relativedelta(months=3)

        if str(train_start_date) == '2017-04-01':
            break
            
    y_data = []
    score_list = []
    feature_list = []

    for i in range(0,len(train_list)):
        X_train_before = train_list[i].drop([y_name,'Cat'],axis =1)
        y_train_before = train_list[i]['Cat']
        X_test = test_list[i].drop([y_name,'Cat'],axis =1)
        y_test = test_list[i]['Cat']
        
        smote = SMOTE(random_state=0,k_neighbors = 3)
        X_train,y_train = smote.fit_resample(X_train_before,y_train_before)
#         ros = RandomOverSampler()
#         X_train,y_train = ros.fit_resample(X_train_before,y_train_before)

       # 랜덤 포레스트 학습 및 별도의 테스트 셋으로 예측 성능 평가
              
        clf =  XGBClassifier(random_state = 0 ,colsample_bytree = 1, gamma =  0.01,\
                            learning_rate =  0.01, max_depth =  4, reg_lambda =  0, subsample =  1,
                             eval_metric = "mlogloss")
        

        clf.fit(X_train , y_train)
        train_pred = clf.predict(X_train_before)
        test_pred = clf.predict(X_test)




        #feature_importance
        feature_importance = clf.feature_importances_


        pred_value = pd.Series(test_pred,index = y_test.index)
        per = test_list[i][y_name]
        y_testdata = pd.concat([per,y_test,pred_value] , axis = 1)
        y_testdata.columns = ['실제 y 수익률','실제 y라벨링','예측 y 라벨링']
        
        train_pred_value = pd.Series(train_pred,index = y_train_before.index)
        per = train_list[i][y_name]
        y_traindata = pd.concat([per,y_train_before,train_pred_value] , axis = 1)
        y_traindata.columns = ['실제 y 수익률','실제 y라벨링','예측 y 라벨링']
        
        
        # 성과 평가
        train_res = get_clf_eval(y_train_before, y_traindata, train_pred)
        test_res = get_clf_eval(y_test, y_testdata,test_pred)
        res = train_res + test_res
        
        score_list.append(res)
    df_score = pd.DataFrame(score_list,columns = ["정확도","정밀도","재현율","f1_score"]+["정확도_test","정밀도_test","재현율_test","f1_score_test"]).T 
    df_score.rename(columns = lambda x : "model_set_"+ str(x),inplace = True)


        
    return df_score

In [16]:
df_score_0_1_xgb = process_final_xgb(df_0_1,'공모가 대비 1개월 수익률')

In [17]:
df_score_0_1_xgb['xgb_mean'] = df_score_0_1_xgb.mean(axis=1)

In [18]:
xgb_score = df_score_0_1_xgb['xgb_mean']

In [19]:
xgb_score

정확도              0.809976
정밀도              0.813935
재현율              0.899144
f1_score         0.815789
정확도_test         0.318096
정밀도_test         0.457703
재현율_test         0.498978
f1_score_test    0.308028
Name: xgb_mean, dtype: float64

# lgbm

In [20]:
def process_final_lgbm(df,y_name):
    
    ## 기본 전처리
    df['상장일'] = pd.to_datetime(df['상장일'])
    df = df.set_index(['상장일'])
    df = df.drop(['종목명','공모 시가총액'],axis = 1) ## 나중에 카테고리 진행할려면 남겨줄 것
    if y_name == '공모가 대비 6개월 수익률' or "종가 대비 6개월 수익률":
        df['Cat'] = Cut(df[y_name],[ -0.2, 0.2, 0.4])
       
        
    else :
        df['Cat'] = Cut(df[y_name],[ -0.1, 0.1, 0.2])
       
    
    ## train 3년 test 1년으로 총 기간을 3개월 이동으로 36개 구간분할
    train_list = []
    test_list = []
    train_start_date = '2009-04-01' ## 기한은 나중에 변경할수도
    test_start_date = parse(str(train_start_date)).date() + relativedelta(years =4)
    train_end_date = parse(str(test_start_date)).date() - relativedelta(days =1)
    test_end_date = parse(str(train_end_date)).date() + relativedelta(years =1) 

    while True:

        train_list.append(df[train_start_date : train_end_date])
        test_list.append(df[test_start_date : test_end_date])

        train_start_date = parse(str(train_start_date)).date() + relativedelta(months=3)
        test_start_date = parse(str(test_start_date)).date() + relativedelta(months=3)
        train_end_date = parse(str(train_end_date)).date() + relativedelta(months=3)
        test_end_date = parse(str(test_end_date)).date() + relativedelta(months=3)

        if str(train_start_date) == '2017-04-01':
            break
            
    y_data = []
    score_list = []
    feature_list = []

    for i in range(0,len(train_list)):
        X_train_before = train_list[i].drop([y_name,'Cat'],axis =1)
        y_train_before = train_list[i]['Cat']
        X_test = test_list[i].drop([y_name,'Cat'],axis =1)
        y_test = test_list[i]['Cat']
        
        smote = SMOTE(random_state=0,k_neighbors = 3)
        X_train,y_train = smote.fit_resample(X_train_before,y_train_before)
#         ros = RandomOverSampler()
#         X_train,y_train = ros.fit_resample(X_train_before,y_train_before)

       # 랜덤 포레스트 학습 및 별도의 테스트 셋으로 예측 성능 평가
              
        clf =  LGBMClassifier(random_state =0, min_data_in_leaf = 50 , num_leaves = 31, reg_alpha = 0.1)


        clf.fit(X_train , y_train)
        train_pred = clf.predict(X_train_before)
        test_pred = clf.predict(X_test)




        #feature_importance
        feature_importance = clf.feature_importances_


        pred_value = pd.Series(test_pred,index = y_test.index)
        per = test_list[i][y_name]
        y_testdata = pd.concat([per,y_test,pred_value] , axis = 1)
        y_testdata.columns = ['실제 y 수익률','실제 y라벨링','예측 y 라벨링']
        
        train_pred_value = pd.Series(train_pred,index = y_train_before.index)
        per = train_list[i][y_name]
        y_traindata = pd.concat([per,y_train_before,train_pred_value] , axis = 1)
        y_traindata.columns = ['실제 y 수익률','실제 y라벨링','예측 y 라벨링']
        
        
        # 성과 평가
        train_res = get_clf_eval(y_train_before, y_traindata, train_pred)
        test_res = get_clf_eval(y_test, y_testdata,test_pred)
        res = train_res + test_res
        
        score_list.append(res)
    df_score = pd.DataFrame(score_list,columns = ["정확도","정밀도","재현율","f1_score"]+["정확도_test","정밀도_test","재현율_test","f1_score_test"]).T 
    df_score.rename(columns = lambda x : "model_set_"+ str(x),inplace = True)


        
    return df_score

In [21]:
df_score_0_1_lgbm = process_final_lgbm(df_0_1,'공모가 대비 1개월 수익률')

[LightGBM] [Warning] min_data_in_leaf is set=50, min_child_samples=20 will be ignored. Current value: min_data_in_leaf=50
[LightGBM] [Warning] min_data_in_leaf is set=50, min_child_samples=20 will be ignored. Current value: min_data_in_leaf=50
[LightGBM] [Warning] min_data_in_leaf is set=50, min_child_samples=20 will be ignored. Current value: min_data_in_leaf=50
[LightGBM] [Warning] min_data_in_leaf is set=50, min_child_samples=20 will be ignored. Current value: min_data_in_leaf=50
[LightGBM] [Warning] min_data_in_leaf is set=50, min_child_samples=20 will be ignored. Current value: min_data_in_leaf=50
[LightGBM] [Warning] min_data_in_leaf is set=50, min_child_samples=20 will be ignored. Current value: min_data_in_leaf=50
[LightGBM] [Warning] min_data_in_leaf is set=50, min_child_samples=20 will be ignored. Current value: min_data_in_leaf=50
[LightGBM] [Warning] min_data_in_leaf is set=50, min_child_samples=20 will be ignored. Current value: min_data_in_leaf=50
[LightGBM] [Warning] min

In [22]:
df_score_0_1_lgbm

,model_set_0,model_set_1,model_set_2,model_set_3,model_set_4,model_set_5,model_set_6,model_set_7,model_set_8,model_set_9,...,model_set_22,model_set_23,model_set_24,model_set_25,model_set_26,model_set_27,model_set_28,model_set_29,model_set_30,model_set_31
정확도,0.934524,0.928571,0.946667,0.943750,0.912752,0.909091,0.884892,0.879195,0.863309,0.821918,...,0.987654,1.000000,1.000000,0.987448,1.000000,0.995708,0.995708,0.987069,0.979839,0.983333
정밀도,0.965517,0.937500,0.940000,0.944444,0.925926,0.942308,0.890909,0.909091,0.873239,0.859155,...,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,0.986301,0.977528,0.978947
재현율,0.918033,0.918367,0.940000,0.927273,0.909091,0.907407,0.890909,0.895522,0.911765,0.835616,...,0.979167,1.000000,1.000000,0.975610,1.000000,0.986111,0.985915,0.972973,0.988636,0.978947
f1_score,0.934677,0.928847,0.946728,0.943850,0.913162,0.908757,0.884958,0.879436,0.863513,0.822388,...,0.987672,1.000000,1.000000,0.987434,1.000000,0.995694,0.995694,0.987032,0.979895,0.983333
정확도_test,0.300000,0.400000,0.305556,0.347826,0.288889,0.321429,0.295082,0.380282,0.467532,0.328358,...,0.403226,0.467742,0.476190,0.416667,0.359375,0.258621,0.202899,0.246154,0.400000,0.473684
정밀도_test,0.750000,0.571429,0.391304,0.593750,0.558824,0.536585,0.523810,0.431373,0.468085,0.340909,...,0.500000,1.000000,0.250000,0.333333,0.450000,0.571429,0.428571,0.681818,0.703704,0.736842
재현율_test,0.176471,0.500000,0.562500,0.730769,0.760000,0.733333,0.687500,0.687500,0.666667,0.576923,...,0.222222,0.166667,0.066667,0.055556,0.333333,0.111111,0.062500,0.333333,0.633333,1.000000
f1_score_test,0.275873,0.337842,0.291686,0.329262,0.267148,0.292153,0.279815,0.354446,0.452675,0.333294,...,0.330949,0.397849,0.367814,0.321254,0.355357,0.163924,0.149723,0.248465,0.370417,0.338346


In [23]:
df_score_0_1_lgbm['lgbm_mean'] = df_score_0_1_lgbm.mean(axis=1)

In [24]:
lgbm_score = df_score_0_1_lgbm['lgbm_mean']

In [25]:
lgbm_score

정확도              0.950360
정밀도              0.962831
재현율              0.951537
f1_score         0.950507
정확도_test         0.338412
정밀도_test         0.477215
재현율_test         0.494482
f1_score_test    0.312700
Name: lgbm_mean, dtype: float64

# 결과값

In [27]:
df = pd.concat([rlf_score,xgb_score,lgbm_score],axis=1)

In [28]:
df

,rlf_mean,xgb_mean,lgbm_mean
정확도,0.702678,0.809976,0.950360
정밀도,0.726528,0.813935,0.962831
재현율,0.805926,0.899144,0.951537
f1_score,0.709760,0.815789,0.950507
정확도_test,0.322484,0.318096,0.338412
정밀도_test,0.476976,0.457703,0.477215
재현율_test,0.558717,0.498978,0.494482
f1_score_test,0.304779,0.308028,0.312700


In [30]:
df.to_csv("모델별 평가지표 비교.csv",encoding = 'euc-kr')

## SMOTE VS ROS

In [37]:
def process_final_ros(df,y_name):
    
    ## 기본 전처리
    df['상장일'] = pd.to_datetime(df['상장일'])
    df = df.set_index(['상장일'])
    df = df.drop(['종목명','공모 시가총액'],axis = 1) ## 나중에 카테고리 진행할려면 남겨줄 것
    if y_name == '공모가 대비 6개월 수익률' or "종가 대비 6개월 수익률":
        df['Cat'] = Cut(df[y_name],[ -0.2, 0.2, 0.4])
       
        
    else :
        df['Cat'] = Cut(df[y_name],[ -0.1, 0.1, 0.2])
       
    
    ## train 3년 test 1년으로 총 기간을 3개월 이동으로 36개 구간분할
    train_list = []
    test_list = []
    train_start_date = '2009-04-01' ## 기한은 나중에 변경할수도
    test_start_date = parse(str(train_start_date)).date() + relativedelta(years =4)
    train_end_date = parse(str(test_start_date)).date() - relativedelta(days =1)
    test_end_date = parse(str(train_end_date)).date() + relativedelta(years =1) 

    while True:

        train_list.append(df[train_start_date : train_end_date])
        test_list.append(df[test_start_date : test_end_date])

        train_start_date = parse(str(train_start_date)).date() + relativedelta(months=3)
        test_start_date = parse(str(test_start_date)).date() + relativedelta(months=3)
        train_end_date = parse(str(train_end_date)).date() + relativedelta(months=3)
        test_end_date = parse(str(test_end_date)).date() + relativedelta(months=3)

        if str(train_start_date) == '2017-04-01':
            break
            
    y_data = []
    score_list = []
    feature_list = []

    for i in range(0,len(train_list)):
        X_train_before = train_list[i].drop([y_name,'Cat'],axis =1)
        y_train_before = train_list[i]['Cat']
        X_test = test_list[i].drop([y_name,'Cat'],axis =1)
        y_test = test_list[i]['Cat']
        
#         smote = SMOTE(random_state=0,k_neighbors = 3)
#         X_train,y_train = smote.fit_resample(X_train_before,y_train_before)
        ros = RandomOverSampler()
        X_train,y_train = ros.fit_resample(X_train_before,y_train_before)

       # 랜덤 포레스트 학습 및 별도의 테스트 셋으로 예측 성능 평가
              
        clf = RandomForestClassifier(random_state=0 , max_depth = 5 , min_samples_leaf = 6 , min_samples_split =  12 , \
                                     n_estimators = 20 )
        

        clf.fit(X_train , y_train)
        train_pred = clf.predict(X_train_before)
        test_pred = clf.predict(X_test)




        #feature_importance
        feature_importance = clf.feature_importances_


        pred_value = pd.Series(test_pred,index = y_test.index)
        per = test_list[i][y_name]
        y_testdata = pd.concat([per,y_test,pred_value] , axis = 1)
        y_testdata.columns = ['실제 y 수익률','실제 y라벨링','예측 y 라벨링']
        
        train_pred_value = pd.Series(train_pred,index = y_train_before.index)
        per = train_list[i][y_name]
        y_traindata = pd.concat([per,y_train_before,train_pred_value] , axis = 1)
        y_traindata.columns = ['실제 y 수익률','실제 y라벨링','예측 y 라벨링']
        
        
        # 성과 평가
        train_res = get_clf_eval(y_train_before, y_traindata, train_pred)
        test_res = get_clf_eval(y_test, y_testdata,test_pred)
        res = train_res + test_res
        
        score_list.append(res)
        feature_list.append(feature_importance)
        y_data.append(y_testdata)   
    df_score = pd.DataFrame(score_list,columns = ["정확도","정밀도","재현율","f1_score"]+["정확도_test","정밀도_test","재현율_test","f1_score_test"]).T 
    df_score.rename(columns = lambda x : "model_set_"+ str(x),inplace = True)


        
    return df_score 

In [38]:
df_score_0_1_ros = process_final_ros(df_0_1,'공모가 대비 1개월 수익률')

In [54]:
df_score_0_1_ros['ros_mean'] = df_score_0_1_ros.mean(axis=1)

In [55]:
ros_score = df_score_0_1_ros['ros_mean']

In [57]:
data = pd.concat([rlf_score,ros_score],axis = 1)

In [60]:
data = data.rename(columns = {'rlf_mean':"smote_mean"})

In [61]:
data

,smote_mean,ros_mean
정확도,0.702678,0.726614
정밀도,0.726528,0.736517
재현율,0.805926,0.835657
f1_score,0.709760,0.730981
정확도_test,0.322484,0.324001
정밀도_test,0.476976,0.469909
재현율_test,0.558717,0.612459
f1_score_test,0.304779,0.308602


In [63]:
data.to_csv("smote_ros 비교.csv",encoding = 'euc-kr')

## 클래스를 3개로 했을때

In [84]:
def process_final_3(df,y_name):
    
    ## 기본 전처리
    df['상장일'] = pd.to_datetime(df['상장일'])
    df = df.set_index(['상장일'])
    df = df.drop(['종목명','공모 시가총액'],axis = 1) ## 나중에 카테고리 진행할려면 남겨줄 것
    if y_name == '공모가 대비 6개월 수익률' or "종가 대비 6개월 수익률":
        df['Cat'] = Cut(df[y_name],[ 0])
       
        
    else :
        df['Cat'] = Cut(df[y_name],[ 0])
       
    
    ## train 3년 test 1년으로 총 기간을 3개월 이동으로 36개 구간분할
    train_list = []
    test_list = []
    train_start_date = '2009-04-01' ## 기한은 나중에 변경할수도
    test_start_date = parse(str(train_start_date)).date() + relativedelta(years =4)
    train_end_date = parse(str(test_start_date)).date() - relativedelta(days =1)
    test_end_date = parse(str(train_end_date)).date() + relativedelta(years =1) 

    while True:

        train_list.append(df[train_start_date : train_end_date])
        test_list.append(df[test_start_date : test_end_date])

        train_start_date = parse(str(train_start_date)).date() + relativedelta(months=3)
        test_start_date = parse(str(test_start_date)).date() + relativedelta(months=3)
        train_end_date = parse(str(train_end_date)).date() + relativedelta(months=3)
        test_end_date = parse(str(test_end_date)).date() + relativedelta(months=3)

        if str(train_start_date) == '2017-04-01':
            break
            
    y_data = []
    score_list = []
    feature_list = []

    for i in range(0,len(train_list)):
        X_train_before = train_list[i].drop([y_name,'Cat'],axis =1)
        y_train_before = train_list[i]['Cat']
        X_test = test_list[i].drop([y_name,'Cat'],axis =1)
        y_test = test_list[i]['Cat']
        
        smote = SMOTE(random_state=0)
        X_train,y_train = smote.fit_resample(X_train_before,y_train_before)
#         ros = RandomOverSampler()
#         X_train,y_train = ros.fit_resample(X_train_before,y_train_before)

       # 랜덤 포레스트 학습 및 별도의 테스트 셋으로 예측 성능 평가
              
        clf = RandomForestClassifier(random_state=0 , max_depth = 5 , min_samples_leaf = 6 , min_samples_split =  12 , \
                                     n_estimators = 20 )
        

        clf.fit(X_train , y_train)
        train_pred = clf.predict(X_train_before)
        test_pred = clf.predict(X_test)




        #feature_importance
        feature_importance = clf.feature_importances_


        pred_value = pd.Series(test_pred,index = y_test.index)
        per = test_list[i][y_name]
        y_testdata = pd.concat([per,y_test,pred_value] , axis = 1)
        y_testdata.columns = ['실제 y 수익률','실제 y라벨링','예측 y 라벨링']
        
        train_pred_value = pd.Series(train_pred,index = y_train_before.index)
        per = train_list[i][y_name]
        y_traindata = pd.concat([per,y_train_before,train_pred_value] , axis = 1)
        y_traindata.columns = ['실제 y 수익률','실제 y라벨링','예측 y 라벨링']
        
        
        # 성과 평가
        train_res = get_clf_eval(y_train_before, y_traindata, train_pred)
        test_res = get_clf_eval(y_test, y_testdata,test_pred)
        res = train_res + test_res
        
        score_list.append(res)
        feature_list.append(feature_importance)
        y_data.append(y_testdata)   
    df_score = pd.DataFrame(score_list,columns = ["정확도","정밀도","재현율","f1_score"]+["정확도_test","정밀도_test","재현율_test","f1_score_test"]).T 
    df_score.rename(columns = lambda x : "model_set_"+ str(x),inplace = True)

    df_feature = pd.DataFrame(feature_list,columns = X_train.columns).T 
    df_feature.rename(columns = lambda x : "model_set_"+ str(x),inplace = True)
        
    return df_score , df_feature , y_data

In [85]:
df_score_3,df_feature_3,y_data_3 = process_final_3(df_0_1,'공모가 대비 1개월 수익률')

In [86]:
df_score_3['by3_mean'] = df_score_3.mean(axis=1)

In [87]:
df3_score = df_score_3['by3_mean']

In [89]:
class_34 = pd.concat([rlf_score,df3_score],axis=1)

In [ ]:
class_34.to_csv("class 비교.csv",)